<a href="https://colab.research.google.com/github/alanntl/SELGO-LITE/blob/alanntl/SLEGO_LITE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SLEGO Project: UNSW CSE PhD Research - Alan Siu Lung Ng

# Clone SLEGO repo and setup the foldersystem

In [5]:
from google.colab import drive
from google.colab import files
import os
import sys
from IPython.display import clear_output

# Path to the directory where the repository is or will be cloned
path = '/content/drive/MyDrive/SLEGO'

if not os.path.exists(path):
    # If it doesn't exist, clone the repository
    !git clone https://github.com/alanntl/SELGO-LITE.git {path}
else:
    # If it exists, you might want to pull updates
    %cd {path}
    !git pull
    print("Updated the existing repository.")

folder_path = '/content/drive/MyDrive/SLEGO/slegospace'

os.chdir(folder_path)
# Now you can reference subfolders and files relative to the top-level folder
dataspace = '/dataspace/'  # This is equivalent to '/content/drive/MyDrive/SLEGO/slegospace/dataspace'
recordspace = '/recordspace/'  # Equivalent to '/content/drive/MyDrive/SLEGO/slegospace/recordspace'
functionspace = '/functionspace/'  # Equivalent to '/content/drive/MyDrive/SLEGO/slegospace/functionspace'
knowledgespace = '/knowledgespace/'  # Equivalent to '/content/drive/MyDrive/SLEGO/slegospace/knowledgespace'

files.view(folder_path)

/content/drive/MyDrive/SLEGO
Already up to date.
Updated the existing repository.


<IPython.core.display.Javascript object>

# Install Libraries on Colab temporarily or install them into google drive

In [6]:
%time
# uncomment below codes if you dont want to save these dependencies on google drive
# !pip install -r /content/drive/MyDrive/SLEGO/requirements.txt

# uncomment below codes if you want to save these dependencies on google drive and reuse it next time
# !pip install --target="/content/drive/MyDrive/Colab Notebooks/SLEGO/my_libraries" -r /content/drive/MyDrive/SLEGO/requirements.txt

sys.path.append("/content/drive/MyDrive/SLEGO/my_libraries")
clear_output()


# Import Libraries
Import all the related libraries

In [7]:
import panel as pn
import inspect
import ast  # For safely evaluating the input string
import re
import importlib
import json
import io
import time
import param
import json
from datetime import datetime
import itertools

# if colab or vscode or jupyter
if 'google.colab' in str(get_ipython()):
    pn.extension(comms='colab')
elif 'vscode' in str(get_ipython()):
    pn.extension(comms='vscode')
elif 'jupyter' in str(get_ipython()):
    pn.extension(comms='jupyter')

pn.extension(sizing_mode = 'stretch_both')

# Select which modules to import

In [12]:
def delete_func_file(func_file_path):
    # Check if the file exists
    if os.path.exists(func_file_path):
        # Delete the file
        os.remove(func_file_path)
        #print(f"File {func_file_path} has been deleted.")
    else:
        print(f"No file named {func_file_path} found.")


# Step 3: Get the list of .py files in the folder
py_files = [file for file in os.listdir(folder_path+functionspace) if file.endswith('.py')]

# New Step: Check if func.py exists and delete it
func_file_path = 'func.py'
delete_func_file(func_file_path)

funcfilecombo = pn.widgets.MultiChoice(name='Select Function',
                                       value=['util.py',
                                              'func_data_preprocss.py',
                                              'func_moving_avg_plot.py',
                                              'llm.py',
                                              'func_eda.py',
                                              'func_uci_dataset.py',
                                              'webscrape.py',
                                              'func_arxiv.py',
                                              'func_backtest.py',
                                              'func_autogluon.py'],
                                       options=py_files, height=200)

# create funcfilecombo_change function
def funcfilecombo_change(event):
    delete_func_file(func_file_path)

    py_files = funcfilecombo.value
    # Step 4 (Modified): Create a new file named func.py in the current repository
    with open(func_file_path, 'w') as func_file:
        # Step 5: Iterate over each .py file and append its content to func.py
        for py_file in py_files:
            file_path = os.path.join(folder_path+functionspace, py_file)
            with open(file_path, 'r') as file:
                func_file.write(file.read() + '\n')
    import func
    importlib.reload(func)


# param watch
funcfilecombo.param.watch(funcfilecombo_change, 'value')

display(funcfilecombo)

# call the event
funcfilecombo_change(None)


MultiChoice(height=200, name='Select Function', options=['webscrape.py', ...], sizing_mode='stretch_width', value=['util.py', 'func_data_pre...])

No file named func.py found.


ModuleNotFoundError: No module named 'openai'

# SLEGO APP

In [ ]:
import func
importlib.reload(func)

funccombo = func._create_multi_select_combobox(func)
input_text = pn.widgets.TextAreaInput(value='', placeholder='input the parameters', height=300)
compute_btn = pn.widgets.Button(name='Compute', height =50,  button_type='primary')
output_text= pn.widgets.TextAreaInput(value='',placeholder= 'Results will be shown here')
savepipe_btn = pn.widgets.Button(name='Save Pipeline', height =50)
pipeline_text= pn.widgets.TextInput(value='', placeholder='Input Pipeline Name', height=50)

global pipeline_dict
pipeline_dict = {}


def get_doc_string(pipeline):
    text = input_text.value
    output=''
    data = ast.literal_eval(text)
    data.keys()
    # loop keys
    for key in data.keys():
        output +='#######'+str(key)+'#######\n'
        try:
            output += eval(f'func.{key}.__doc__')+'\n'
        except:
            output += 'No docstring found for this function\n'
    return output

def input_text_change(event):
    text = input_text.value
    text = re.sub(r'\bfalse\b', 'False', input_text.value, flags=re.IGNORECASE)
    pipeline_dict = ast.literal_eval(text)

def funccombo_change(event):
    output_text.value = ''
    list_funcs = funccombo.value
    list_params =[]

    for funcchoice in funccombo.value:
      function=  eval('func.'+funcchoice)
      list_params.append(func._extract_parameter(function))

    funcs_params = dict(zip(list_funcs,list_params))
    formatted_data = json.dumps(funcs_params, indent=5)
    input_text.value = str(formatted_data)
    output_text.value = get_doc_string(input_text.value)


def save_record(space, data, pipeline_name=None):
    """
    Saves the given data structure as a JSON file named with the current datetime.

    Parameters:
    - data: The data structure to be saved as JSON.
    """
    if pipeline_name is None:
      # Generate a filename based on the current datetime, e.g., "record_20230405_153000.json"
      filename = datetime.now().strftime("record_%Y%m%d_%H%M%S.json")
    else:
      filename = pipeline_name+'.json'

    # Define the full path where you want to save the file, here assuming current directory
    full_path = './'+space+'/' + filename

    # Serialize and save the data structure to a file
    with open(full_path, "w") as file:
        json.dump(data, file, indent=5)


def compute_btn_clicked(event):
    input_text_change(None)
    output_text.value = ''
    # Get the text from the text_input
    text = input_text.value
    text = re.sub(r'\bfalse\b', 'False', input_text.value, flags=re.IGNORECASE)


    # You can now do something with the text, like processing it or displaying results
    # For this example, let's just print it (or display it somewhere in your panel app)
    pipeline_dict = ast.literal_eval(text)


    for function_name, parameters in pipeline_dict.items():
        module = 'func.'
        start_time = time.time()  # Start the timer

        # Dynamically execute the function
        function = eval(module + function_name)
        result = function(**parameters)
        # Convert result to string
        result_string = str(result)
        # Create an iterator for words
        words_iterator = iter(result_string.split())
        # Use itertools.islice to get the first 1000 words without creating a full list
        first_x_words = itertools.islice(words_iterator, 500)
        # Join the words and append to output_text.value
        # Computation time
        compute_time = time.time() - start_time

        output_text.value += f"\n===================={function_name}====================\n\n"
        output_text.value += f"Function computation Time: {compute_time:.4f} seconds\n"
        output_text.value += f"\n"
        output_text.value += " ".join(first_x_words)

    save_record('recordspace', pipeline_dict)

def save_pipeline(event):
  if pipeline_text.value == '':
    pipeline_name = '__'
  else:
    pipeline_name = pipeline_text.value
  text = input_text.value
  text = re.sub(r'\bfalse\b', 'False', input_text.value, flags=re.IGNORECASE)
  data = ast.literal_eval(text)
  save_record('knowledgespace', data, pipeline_name)

# save output into specific folder
funccombo.param.watch(funccombo_change, 'value')
input_text.param.watch(input_text_change, 'value')
compute_btn.on_click(compute_btn_clicked)
savepipe_btn.on_click(save_pipeline)
app = pn.Row(pn.Column(funccombo,compute_btn,pn.Row(savepipe_btn,pipeline_text), input_text), pn.Column(output_text))

# Display the App

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, message='.*findfont.*')
# Creating a Panel column to display the combobox and the text area
# sk-ThBFYGul5uQJN13tHYgkT3BlbkFJNrFzdHY368549fywvav
app

Row(sizing_mode='stretch_both')
    [0] Column(sizing_mode='stretch_both')
        [0] MultiChoice(height=150, name='Functions:', options=['autogluon_model_predict'...], sizing_mode='stretch_width')
        [1] Button(button_type='primary', height=50, name='Compute', sizing_mode='stretch_width')
        [2] Row(sizing_mode='stretch_both')
            [0] Button(height=50, name='Save Pipeline', sizing_mode='stretch_width')
            [1] TextInput(height=50, placeholder='Input Pipeline Name', sizing_mode='stretch_width')
        [3] TextAreaInput(height=300, placeholder='input the parameters', sizing_mode='stretch_width')
    [1] Column(sizing_mode='stretch_both')
        [0] TextAreaInput(placeholder='Results will b..., sizing_mode='stretch_both')